In [1]:
import re
import time
import pickle
from termcolor import colored
import string
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, TweetTokenizer
from nltk.corpus import words, stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.metrics.distance  import edit_distance, jaccard_distance
from nltk.util import ngrams

In [2]:
Text2 = "loveli weathor today rigt? Hope it stays sanny all doy."
correct_words = words.words()
stop_words = stopwords.words('english')

*               Detecting wrong words

In [3]:
def detect_mistakes(text):
    tk = TweetTokenizer()
    tokens = tk.tokenize(Text2)
    tokens_list = []
    Result_text = ""
    for word in tokens:
        if len(word) <= 1:
            tokens_list.append(word)
        elif (len(word) > 1 and word.lower() in correct_words) or ((len(word) > 1 and word.lower() in stop_words)):
            tokens_list.append(word)
        else:
            tokens_list.append(colored(word, 'red'))
    Result_text = " ".join([str(word) for word in tokens_list])
    ponc_with_space = [' , ',' ? ',' ! ']
    ponc_with_nospace = [', ','? ','! ']
    for i in range(len(ponc_with_space)):
        Result_text = Result_text.replace(ponc_with_space[i],ponc_with_nospace[i])
    print(Result_text)

In [4]:
detect_mistakes(Text2)

loveli weathor today rigt? Hope it stays sanny all doy .


*               Correcting words

In [5]:
def correct_mistakes(text):
    cor_words = {}
    tk = TweetTokenizer()
    tokens = tk.tokenize(text)
    incorrect_words = []
    for word in tokens:
        if ((len(word) > 1 and word.lower() not in correct_words) and ((len(word) > 1 and word.lower() not in stop_words))):
            incorrect_words.append(word)
    print("---------------------------------------------------------------------------")
    print("In case, you don't get the wanted correction. Please type it. Else type no.")
    print("---------------------------------------------------------------------------")
    for word in incorrect_words:
        
        temp = [(edit_distance(word, w),w) for w in correct_words if w[0]==word[0]]
        temp2 = [(jaccard_distance(set(ngrams(word, 2)),set(ngrams(w, 2))),w) for w in correct_words if w[0]==word[0]]

        a = sorted(temp, key = lambda val:val[0])[0][1]
        b = sorted(temp2, key = lambda val:val[0])[0][1]
        
        if a == b:
            cor_words[word]=[a]
            print(colored(word, 'red') , " : ", a ,end = "")
        else:
            cor_words[word]=[a,b]
            print(colored(word, 'red') , " : ", a, " | ", b, end = "")
        
        time.sleep(1)

        message = input("Add the wanted correction :")

        if message != "no":
            print(" | ",message)
            cor_words[word].append(message)
            
        else:
            print()
    print("---------------------------------------------------------------------------")
    pickle.dump(cor_words, open('./cor_words.pkl', 'wb'))
    pickle.dump(incorrect_words, open('./incorrect_words.pkl', 'wb'))

detect_mistakes(Text2) 
correct_mistakes(Text2)

loveli weathor today rigt? Hope it stays sanny all doy .
---------------------------------------------------------------------------
In case, you don't get the wanted correction. Please type it. Else type no.
---------------------------------------------------------------------------
loveli  :  lovely  |  lovelily
weathor  :  weather
rigt  :  rift  |  rig |  right
sanny  :  sandy  |  sannyasi |  sunny
doy  :  day  |  do
---------------------------------------------------------------------------


*               select the wanted word

In [10]:
def select_word():
    cor_words = pickle.load(open('./cor_words.pkl','rb'))
    chosen_words = []
    for key, val in cor_words.items():
        print(f"For the word {key}, please type the correct word among the following words : {val}.")
        time.sleep(1)
        message = input("You :")
        chosen_words.append(message)
    pickle.dump(chosen_words, open('./chosen_words.pkl', 'wb'))

select_word()

For the word loveli, please type the correct word among the following words : ['lovely', 'lovelily'].
For the word weathor, please type the correct word among the following words : ['weather'].
For the word rigt, please type the correct word among the following words : ['rift', 'rig', 'right'].
For the word sanny, please type the correct word among the following words : ['sandy', 'sannyasi', 'sunny'].
For the word doy, please type the correct word among the following words : ['day', 'do'].


*               Update Text

In [9]:
def final_text(text):
    incorrect_words = pickle.load(open('./incorrect_words.pkl','rb'))
    chosen_words = pickle.load(open('./chosen_words.pkl','rb'))
    for i in range(len(chosen_words)):
        text = text.replace(incorrect_words[i],chosen_words[i])
    return text

final_text(Text2)

'lovely weather today right? Hope it stays sunny all da.'